In [3]:
import pandas as pd
import numpy as np
from collections import namedtuple
import transformers
from transformers import (Trainer, TrainingArguments, AutoTokenizer , AutoModel, pipeline, 
                          DataCollatorWithPadding, TrainerCallback, AutoModelForSequenceClassification, EvalPrediction)
from datasets import load_dataset, Dataset, load_metric
import torch
import torch.nn.functional as F
from torch import utils
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, 
                             classification_report, confusion_matrix)
import mlflow
from mlflow.tracking import MlflowClient
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold


2023-11-06 16:14:48.126859: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-06 16:14:48.434100: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-06 16:14:48.434193: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-06 16:14:48.435244: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-06 16:14:48.554877: I tensorflow/core/platform/cpu_feature_g

In [4]:
#setting fixed seeds
def set_seeds(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
set_seeds(42)


In [5]:
#utils
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, model_name, file_path, text_field, label_field, years, seq_len):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        df = pd.read_csv(file_path)
        df = df.loc[df['Fiscal Year'].isin(years)]
        self.encoding = {}
        for field, seq_len in zip(text_field, seq_len):
            df[field].fillna('No text available', inplace = True)
            self.texts = df[field].tolist()
            if field == 'Specific Aims':
                field_encoding = self.tokenizer(self.texts, 
                                    max_length=seq_len,
                                    padding='max_length',
                                    return_tensors='pt',
                                    truncation=True,
                                    truncation_strategy = 'longest_first')
            else:
                field_encoding = self.tokenizer(self.texts, 
                                    max_length=seq_len,
                                    padding='max_length',
                                    return_tensors='pt',
                                    truncation=True)
            if self.encoding == {}:
                self.encoding = field_encoding
            else:
                for key, val in self.encoding.items():
                    self.encoding[key] = torch.cat((self.encoding[key], field_encoding[key]), dim = 1)
                    
        if df[label_field].isna().any():
        #df[label_field].fillna('No label available', inplace = True) 
            print('label not available')
            
        self.labels = df[label_field].replace({0: 1, 1: 0}).tolist()
        self.appl_id = df['Appl ID'].tolist()
        self.activity_code = df['Activity Code'].tolist()
    
    def __len__(self):
        return len(self.texts)
        
    def __getitem__(self, idx):
        
        item = {key: torch.tensor(val[idx]) for key, val in self.encoding.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
            
        item['Appl ID'] = torch.tensor(self.appl_id[idx])
        item['Activity Code'] = self.activity_code[idx]
        return item
    
    
def compute_metrics_fn(p: EvalPrediction):
        predictions = p.predictions
        label_ids = p.label_ids

        logits = predictions
        preds = np.argmax(logits, axis=-1)

        report = classification_report(label_ids, preds)
        cm = confusion_matrix(label_ids, preds)
        print(report)
        print('confusion matrix: ', cm)

        metrics = {}
        accuracy = accuracy_score(label_ids, preds)
        precision = precision_score(label_ids, preds)
        recall = recall_score(label_ids, preds)
        f1 = f1_score(label_ids, preds)

        metrics['accuracy'] = accuracy
        metrics['precision'] = precision
        metrics['recall'] = recall
        metrics['eval_f1'] = f1

        return metrics
        
      

In [6]:
class CustomizedTrainer(Trainer):
   
    
#     def compute_loss(self, model, inputs, return_outputs=False, loss_fn = 'class_weight'):
#         if loss_fn == 'class_weight':
#             return self.compute_weight_adjusting_loss(model, inputs, return_outputs)
#         elif loss_fn == 'focal':
#             return self.compute_focal_loss(model, inputs, return_outputs)
#         else:
#             raise ValueError("Invalid loss function specified")
            
    def compute_loss(self, model, inputs, return_outputs=False):
        target = inputs.get('labels')
        
        labels = torch.unique(target)
        
        class_distribution = torch.zeros(len(labels))
        
        for i in labels:
            class_count = torch.count_nonzero(target == i)
            class_distribution[i] = class_count
            
        #weights = (target.shape[0] / class_distribution).to(model.device)
        weights = torch.where(class_distribution > 0, target.shape[0] / class_distribution, torch.zeros_like(class_distribution)).to(model.device)

        
        criterion = torch.nn.CrossEntropyLoss(weight = weights)
        outputs = model(**inputs)
        logits = outputs.get('logits')
        prob = torch.nn.functional.softmax(logits, dim = -1)
        
        loss = criterion(prob.to(target.device), target)
        #gradients = [p.grad for p in model.parameters() if p.grad is not None]
        #print('Gradients are:', gradients)
        
        return (loss, outputs) if return_outputs else loss
     
    #focal loss
#     def compute_loss(self, model, inputs, return_outputs=False):
#         #0.8 for prevention and 0.2 for non-prevention
#         alpha = [0.55, 0.45]
#         gamma = 2
#         target = inputs.get('labels')
#         labels = torch.unique(target)
        
#         outputs = model(**inputs)
#         logits = outputs.get('logits')
#         #print(logits)
#         prob = torch.nn.functional.log_softmax(logits, dim = -1)
        
#         #print(prob, target)
        
#         logpt = prob.gather(1,target.view(-1, 1))
#         logpt = logpt.view(-1)
#         pt = logpt.exp()
        
#         alpha = torch.tensor(alpha, device = model.device)
#         if alpha is not None:
#             #if alpha.type()!=inputs.data.type():
#                 #alpha = alpha.type_as(inputs.data)
#             at = alpha.gather(0,target.data.view(-1))
#             logpt = logpt * at

#         focal_loss = -1 * (1-pt)**gamma * logpt
#         loss = focal_loss.sum() / len(focal_loss)
#         #gradients = [p.grad for p in model.parameters() if p.grad is not None]
#         #print('Gradients are:', gradients)
#         return (loss, outputs) if return_outputs else loss

In [7]:
torch.manual_seed(42)
class PreventionTrainer():
    def __init__(self,  model_name, output_dir, batch_size, learning_rate, weight_decay, num_epochs, lr_scheduler,
                run_name, resume_checkpoint):
        self.output_dir = output_dir
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.weight_decay = weight_decay
        self.num_epochs = num_epochs
        self.lr_scheduler = lr_scheduler
        self.run_name = f"{run_name}_{self.learning_rate}"
        self.resume_checkpoint = resume_checkpoint
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.datacollator = DataCollatorWithPadding(tokenizer = self.tokenizer)
        
    def verify_initial_loss(self):
        n_classes = self.model.config.num_labels
        expected_initial_loss = - torch.log(torch.tensor(1.0 / n_classes))
        
        dummy_input = torch.randn(2, self.batch_size)
        dummy_target = torch.zeros(1, dtype=torch.long)
        
        
        loss = compute_loss(self.model, dummy_input, return_outputs=False)
        
        if torch.allclose(loss, expected_initial_loss):
            print("Initial loss verification successful!")
        else:
            print("Initial loss verification failed.")
            print(f"Expected initial loss: {expected_initial_loss}, Computed initial loss: {loss}")
       
        
    def _create_training_args(self):
        training_args = TrainingArguments(
            output_dir = self.output_dir,
            evaluation_strategy = 'epoch',
            per_device_train_batch_size = self.batch_size,
            per_device_eval_batch_size = self.batch_size,
            learning_rate = self.learning_rate,
            weight_decay = self.weight_decay,
            num_train_epochs = self.num_epochs,
            #lr_scheduler_type = self.lr_scheduler,
            log_level = 'debug',
            logging_strategy = 'steps',
            save_strategy = 'epoch',
            save_total_limit = 1,
            seed = 42,
            fp16 = True,
            run_name = self.run_name,
            #label_names = ['prevention', 'non_prev'],
            load_best_model_at_end = True,
            metric_for_best_model = "eval_f1",
            label_smoothing_factor = 0.2,
            debug = "underflow_overflow",
            optim = 'adamw_hf',
            #group_by_length = True,
            report_to = 'mlflow',
            #set when needed
            resume_from_checkpoint = self.resume_checkpoint,
            gradient_checkpointing = True )
        
        return training_args
        

    
    def train(self, train_dataset, eval_dataset):
        with mlflow.start_run():
            # Log training parameters
            mlflow.log_param("learning_rate", self.learning_rate)
            mlflow.log_param("batch_size", self.batch_size)
            mlflow.log_param("num_epochs", self.num_epochs)
            print("===> Loading Model and Training Arguments...")

            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            self.model.to(device)

            training_args = self._create_training_args()

            print("===> Predicting...")

            trainer = Trainer(
                model = self.model,
                args = training_args,
                data_collator = self.datacollator,
                train_dataset = train_dataset,
                eval_dataset = eval_dataset,
                compute_metrics = compute_metrics_fn)

#             def track_gradients(module, grad_input, grad_output):
#                 for param_name, grad in module.named_parameters():
#                     if grad is not None:
#                         print(f'Layer: {param_name}, Mean Grad: {grad.mean().item()}, Max Grad: {grad.max().item()}')

            # Attach the hook to each module's gradient computation
    #         for module in self.model.modules():
    #             module.register_backward_hook(track_gradients)

            trainer.train()
       
        mlflow.end_run()
        
        
    def train_kfold(self, dataset):
        print("===> Loading Model and Training Arguments...")

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)

        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        fold = 1
        for train_idx, eval_idx in kfold.split(dataset):
            # Set the experiment path
            experiment_path = "BioLinkClassifier"
            # Set the experiment
            mlflow.set_experiment(experiment_path)
            with mlflow.start_run():
                # Log training parameters
                mlflow.log_param("learning_rate", self.learning_rate)
                mlflow.log_param("batch_size", self.batch_size)
                mlflow.log_param("num_epochs", self.num_epochs)
                print(f"Training on Fold {fold}")
                train_fold = [dataset[i] for i in (train_idx)]
                eval_fold = [dataset[i] for i in (eval_idx)]

                training_args = self._create_training_args()

                print("===> Predicting...")

                trainer = Trainer(
                    model = self.model,
                    args = training_args,
                    data_collator = self.datacollator,
                    train_dataset = train_fold,
                    eval_dataset = eval_fold,
                    compute_metrics = compute_metrics_fn    

                )

                trainer.train()
                fold += 1
                #self.run_name += str(fold)

            mlflow.end_run()
            
    def evaluate_heldout(self, test_dataset, model_checkpoint):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels = 2)

        training_args = self._create_training_args()

        print("===> Performance on held-out set...")

        trainer = Trainer(
            model = model,
            args = training_args,
            data_collator = self.datacollator,
            eval_dataset = test_dataset,
            compute_metrics = compute_metrics_fn)
        
        ModelOutput = namedtuple(
                        "ModelOutput",
                        ["predictions",  
                            "label_ids",   
                            "metrics" ]
                    )
        print('predictions on test set:')
        logits = trainer.predict(test_dataset).predictions
        probabilities = F.softmax(torch.tensor(logits), dim=1)
        probabilities = probabilities.numpy()

        df = pd.DataFrame(probabilities, columns=['0', '1'])
        df['predicted_class'] = df.apply(lambda row: 0 if row['0'] > row['1'] else 1, axis=1)
        df['Appl ID'] = [item["Appl ID"].item() for item in test_dataset]
        df['True Label'] = [item['labels'].item() for item in test_dataset]
        df['Activity Code'] = [item["Activity Code"] for item in test_dataset]
        
        df.to_csv('LM_predictions.csv', index = False)
        print(trainer.predict(test_dataset).predictions)
        
        
               

In [22]:
def main():
    
    #prepare training dataset
    train_dataset = CustomDataset('michiyasunaga/BioLinkBERT-base', 
                                  #12 activity code
                                 #'ODP_FY12-21_funded_20230815_Joined.csv', 
                                  #'2021coded_2023-06-22.csv',
                                  #all ACs
                                  'ODP_fy1221-2023-11-01.csv',
                                  ['Title', 'Abstract', 'Public Health Relevance'], 
                                  'f6otherorunclear', 
                                  [2017, 2018, 2019, 2020, 2021], 
                                  [35, 221, 256])
    
  
    
    
#     eval_dataset = CustomDataset('michiyasunaga/BioLinkBERT-base', 
#                                  'ODP_FY12-21_funded_20230815_Joined.csv', 
#                                   ['Title', 'Abstract', 'Specific Aims'], 
#                                   'f6otherorunclear', 
#                                   [2021], 
#                                   [35, 221, 256])

    eval_dataset = CustomDataset('michiyasunaga/BioLinkBERT-base',
                                 #'2021coded_12ac_2023-08-15.csv',
                                 #"2021coded_2023-06-22.csv",
                                 #'ODP_fy1221-2023-11-01.csv',
                                      #4300 examples
                                #'2022coded_2023-08-16.csv', 
                                      #7500 examples
                                  #'FY2022_09-28-2023.csv',
                                      #2022 all data with activity codes
                                  'ODP_fy22-all_codes_2023-11-01.csv',
                                 ['Title', 'Abstract', 'Public Health Relevance'], 
                                 #'f6otherorunclear', 
                                 'X 1 Not Codable',
                                 [2022], 
                                 [35, 221, 256])
    #eval_dataset, test_dataset = train_test_split(eval_test_dataset, test_size=0.5, random_state=42)
    
# # # #     train_dataset, eval_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
    
   

       #set experiment params
    #candidate params for 2022
    model_name ='michiyasunaga/BioLinkBERT-base'
    output_dir = 'prevention_finetuned_2020'
    batch_size = 32
    learning_rate = 3e-5
    weight_decay = 1e-5
    num_epochs = 5
    lr_scheduler = 'linear'
    run_name = 'prev_biolink_finetune_19_20'
    resume_checkpoint = None
        
    
    
   
    
    
    
    obj = PreventionTrainer(model_name, output_dir, batch_size, learning_rate, weight_decay, num_epochs, lr_scheduler,run_name, resume_checkpoint)
    
    
# #     #train without cross validation
    obj.train(train_dataset = train_dataset, eval_dataset = eval_dataset)
    
    #preparing test dataset
#     eval_test_dataset = CustomDataset('michiyasunaga/BioLinkBERT-base',
#                                  #'2021coded_12ac_2023-08-15.csv',
#                                  #"2021coded_2023-06-22.csv",
#                                '2022coded_2023-08-16.csv', 
#                                  ['Title', 'Abstract', 'Specific Aims'], 
#                                  #'f6otherorunclear', 
#                                  'X 1 Not Codable',
#                                  [2022], 
#                                  [35, 221, 256])
    
#     #evaluate performance on unseen test data
#     test_dataset = CustomDataset('michiyasunaga/BioLinkBERT-base',
#                                  #'2021coded_12ac_2023-08-15.csv',
#                                  #"2021coded_2023-06-22.csv",
#                                #'2022coded_2023-08-16.csv',
#                                  'FY2022_09-28-2023.csv',
#                                  ['Title', 'Abstract', 'Specific Aims'], 
#                                  #'f6otherorunclear', 
#                                  'X 1 Not Codable',
#                                  [2022], 
#                                  [35, 221, 256])
    obj.evaluate_heldout(eval_dataset, 'prevention_finetuned_2020/checkpoint-1448')

    
    #training with kfold
#     dataset = CustomDataset('michiyasunaga/BioLinkBERT-base', 
#                              'ODP_FY12-21_funded_20230815_Joined.csv',
#                             #'2022coded_2023-08-16.csv',
#                             ['Title', 'Abstract', 'Specific Aims'], 
#                             'f6otherorunclear', 
#                             #'X 1 Not Codable',
#                             [2021], 
#                             [35, 221, 256])
    #split dataset into training and evalkuation for k-fold and a held-out set
    
    #train_eval_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)
    #training the model
    #obj.train_kfold(dataset = train_eval_dataset)
    #obj.evaluate_heldout(test_dataset, 'prevention_finetuned/checkpoint-60')
#     test_dataset2 = CustomDataset('michiyasunaga/BioLinkBERT-base', 
#                                   '2022coded_2023-08-16.csv', 
#                                   ['Title', 'Abstract', 'Specific Aims'], 
#                                   'X 1 Not Codable', 
#                                   [2022], 
#                                   [35, 221, 256])
    #obj.evaluate_heldout(test_dataset2, 'prevention_finetuned/checkpoint-60')


In [23]:
if __name__ == '__main__':
    main()

loading file vocab.txt from cache at /home/elaheh/.cache/huggingface/hub/models--michiyasunaga--BioLinkBERT-base/snapshots/b71f5d70f063d1c8f1124070ce86f1ee463ca1fe/vocab.txt
loading file tokenizer.json from cache at /home/elaheh/.cache/huggingface/hub/models--michiyasunaga--BioLinkBERT-base/snapshots/b71f5d70f063d1c8f1124070ce86f1ee463ca1fe/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /home/elaheh/.cache/huggingface/hub/models--michiyasunaga--BioLinkBERT-base/snapshots/b71f5d70f063d1c8f1124070ce86f1ee463ca1fe/special_tokens_map.json
loading file tokenizer_config.json from cache at /home/elaheh/.cache/huggingface/hub/models--michiyasunaga--BioLinkBERT-base/snapshots/b71f5d70f063d1c8f1124070ce86f1ee463ca1fe/tokenizer_config.json
loading file vocab.txt from cache at /home/elaheh/.cache/huggingface/hub/models--michiyasunaga--BioLinkBERT-base/snapshots/b71f5d70f063d1c8f1124070ce86f1ee463ca1fe/vocab.txt
loading file token

===> Loading Model and Training Arguments...
===> Predicting...


Currently training with a batch size of: 32
/home/elaheh/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 28,794
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 4,500
  Number of trainable parameters = 108,234,242
MLflow experiment_name=None, run_name=prev_biolink_finetune_19_20_3e-05, nested=False, tags=False
/tmp/ipykernel_13014/1676292039.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 

Epoch,Training Loss,Validation Loss,F1,Accuracy,Precision,Recall
1,0.533000,0.494734,0.737605,0.844450,0.659514,0.836672
2,0.478700,0.520224,0.724266,0.829956,0.628399,0.854648
3,0.435200,0.525356,0.724906,0.832640,0.635344,0.843862
4,0.398600,0.563805,0.720601,0.820158,0.606529,0.887519
5,0.372600,0.588790,0.711957,0.815058,0.600282,0.874679


***** Running Evaluation *****
  Num examples = 7451
  Batch size = 32
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: Activity Code, Appl ID. If Activity Code, Appl ID are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.


...

              precision    recall  f1-score   support

           0       0.95      0.80      0.87      5504
           1       0.61      0.89      0.72      1947

    accuracy                           0.82      7451
   macro avg       0.78      0.84      0.79      7451
weighted avg       0.86      0.82      0.83      7451

confusion matrix:  [[4383 1121]
 [ 219 1728]]


Saving model checkpoint to prevention_finetuned_2020/checkpoint-3600
Configuration saved in prevention_finetuned_2020/checkpoint-3600/config.json
Model weights saved in prevention_finetuned_2020/checkpoint-3600/pytorch_model.bin
Deleting older checkpoint [prevention_finetuned_2020/checkpoint-2700] due to args.save_total_limit
/tmp/ipykernel_13014/1676292039.py:43: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encoding.items()}
/home/elaheh/anaconda3/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_ree

              precision    recall  f1-score   support

           0       0.95      0.79      0.86      5504
           1       0.60      0.87      0.71      1947

    accuracy                           0.82      7451
   macro avg       0.77      0.83      0.79      7451
weighted avg       0.86      0.82      0.82      7451

confusion matrix:  [[4370 1134]
 [ 244 1703]]


Saving model checkpoint to prevention_finetuned_2020/checkpoint-4500
Configuration saved in prevention_finetuned_2020/checkpoint-4500/config.json
Model weights saved in prevention_finetuned_2020/checkpoint-4500/pytorch_model.bin
Deleting older checkpoint [prevention_finetuned_2020/checkpoint-3600] due to args.save_total_limit


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from prevention_finetuned_2020/checkpoint-900 (score: 0.7376047090785601).
Deleting older checkpoint [prevention_finetuned_2020/checkpoint-4500] due to args.save_total_limit


OSError: prevention_finetuned_2020/checkpoint-1448 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`